# Nétoyage du jeu de données

In [1]:
import pandas as pd 
import numpy as np

In [2]:
# Chemin vers le fichier csv contenant les données à explorer.
path = "data/original/mower_market_snapshot.csv"

# Chemin vers le répertoire où seront stockées les données nétoyées.
path_out = "data/cleaned/mower_market_snapshot_cleaned.csv"

# Délimiteur utilisé dans le fichier csv
delimiter=";"

# Toutes les valeurs à reconnaître comme des nan.
na_values = ['','unknown',-64]

# Création de la dataframe à partir du fichier csv
mower_df = pd.read_csv(path, delimiter=delimiter, na_values=na_values)

## 1. Travail sur la variable "coût de production"

Nous avons déjà filtré les valeurs abérantes lors du chargement de la dataframe : "", "unknown" ou encore -64. Nous allons dans un premier temps remplacer tous les "nan" par la valeur 0.

In [3]:
# On remplace tous les "nan" par 0
mower_df.prod_cost.fillna(value=0.0, inplace=True)

Maintenant, il sagit de remplacer cette valeure 0, qui reste une valeur abérante. L'approche que nous avons séléctionnée consiste à appliquer la médiane (moins sensible aux valeurs extrêmes) à chacune des valeur nulle. Cependant, nous n'allons pas utiliser la médiane générale mais plutôt la médiane correpondant à chaque **type de produit** pour chaque **indice de qualité**.

In [4]:
# On regroupe la dataframe initiale par qualité et par type de produit
mower_by_quality_by_type  = mower_df[mower_df.prod_cost != 0].groupby(by=["quality", "product_type"])

# medianes pour chaque type, pour chaque indice de qualité
median = mower_by_quality_by_type.prod_cost.median()

# dictionnaire à appliquer que la variable "coût de production"
median_dict = {(-1, 1): None,
               (-1, 2): None,
               (-1, 3): None,
               (0, 1): None,
               (0, 2): None,
               (0, 3): None,
               (1, 1): None,
               (1, 2): None,
               (1, 3): None
              }

for key in mower_by_quality_by_type.prod_cost.groups.keys():
    median_dict[key] = np.median(mower_by_quality_by_type.prod_cost.get_group(key))

In [5]:
# Modifications des valeurs nulles pour la variable "coût de production"
key = (None, None)
for line in range(mower_df.prod_cost.size):
    key = (mower_df["quality"][line], mower_df["product_type"][line])
    prod_cost = mower_df["prod_cost"][line]
    
    if prod_cost == 0:
        mower_df.set_value(line, "prod_cost", median_dict[key])

In [6]:
# On s'assure que la modification permet d'avoir une variable "coût de production cohérente".
mower_df.describe(include='all')

,capacity,failure_rate,id,margin,price,prod_cost,product_type,quality,warranty,market_share,attractiveness
count,1399.000000,1399.000000,1399.000000,1399.000000,1399.000000,1399.000000,1399,1399,1399,1399.000000,1399.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,3,3,27,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,essence,Low,1 an.,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,844,971,103,NaN,NaN
mean,47.663266,0.166448,18958.000000,186.262609,260.075165,57.821619,NaN,NaN,NaN,0.002144,0.636507
std,19.840501,0.073605,404.000825,191.011102,259.949127,60.372861,NaN,NaN,NaN,0.001790,0.124016
min,0.013884,0.028906,18259.000000,52.943989,62.162200,7.095443,NaN,NaN,NaN,0.000564,0.297384
25%,35.853890,0.095678,18608.500000,62.247602,73.379425,10.856096,NaN,NaN,NaN,0.001135,0.549369
50%,50.309242,0.199796,18958.000000,66.299672,78.439753,12.220196,NaN,NaN,NaN,0.001399,0.641351
75%,61.905015,0.228011,19307.500000,265.370973,384.265263,113.929985,NaN,NaN,NaN,0.002446,0.726746


## 2. Travail sur la variable "capacité"

Dans l'étape précédente, nous avons remarqué que la variable "capacité" était parfois inférieure à 1. Ceci va à l'encontre de la dimension supposée (Litrage). Une valeur inférieur à 1, dans notre cas, signifie probablement que la capacité a été insérée en $m^{3}$. Pour s'en assurer, il faudrait revérifier avec le métier.

In [7]:
for line in range(mower_df.prod_cost.size):
    capacity = mower_df["capacity"][line]
    
    if capacity < 1.:
        # Convertion en litres des données en m^3
        mower_df.set_value(line, "capacity", capacity * 1000)

## 3. Travail sur la variable "warranty"

Enfin, nous allons uniformiser la variable "warranty". Pour plus de confort, nous allons la passer en variable entière et l'uniformiser.

In [8]:
for line in range(mower_df.warranty.size):
    warranty = mower_df["warranty"][line]
    mower_df.set_value(line, "warranty", np.int(warranty[0]))

## 4. Corrélations

Avant de sauvegarder l'ensemble de données, nous allons étudier les différentes corrélations entre les variables pour voir si certaines ne peuvent pas être retirées. 

### 4.1. Corrélations des variables continues

In [9]:
mower_df.corr(method='spearman')

,capacity,failure_rate,id,margin,price,prod_cost,market_share,attractiveness
capacity,1.000000,0.674601,-0.042423,-0.624536,-0.571099,-0.653885,-0.568004,0.041825
failure_rate,0.674601,1.000000,-0.020989,-0.764890,-0.697592,-0.761458,-0.740422,-0.133928
id,-0.042423,-0.020989,1.000000,0.040893,0.030357,0.043071,0.026769,-0.002569
margin,-0.624536,-0.764890,0.040893,1.000000,0.900238,0.795176,0.593724,-0.128454
price,-0.571099,-0.697592,0.030357,0.900238,1.000000,0.801597,0.526014,-0.154967
prod_cost,-0.653885,-0.761458,0.043071,0.795176,0.801597,1.000000,0.664326,-0.003017
market_share,-0.568004,-0.740422,0.026769,0.593724,0.526014,0.664326,1.000000,0.606951
attractiveness,0.041825,-0.133928,-0.002569,-0.128454,-0.154967,-0.003017,0.606951,1.000000


On remarque que certaines variable sont fortement corrélées aux autres : 

* la marge
* la part de marché
* le coût de production

Nous allons donc les exclures pour la partie modélisation.

In [10]:
mower_df.drop(labels="margin", axis=1, inplace=True)
mower_df.drop(labels="market_share", axis=1, inplace=True)
mower_df.drop(labels="prod_cost", axis=1, inplace=True)

### 4.2 Corrélations des variables catégorielles

#### 4.2.1. Product type & warranty

In [11]:
mower_by_type_by_warranty = mower_df.groupby(["product_type", "warranty"])

In [12]:
mower_by_type_by_warranty.count()

,,capacity,failure_rate,id,price,quality,attractiveness
product_type,warranty,,,,,,
auto-portee,3,142,142,142,142,142,142
electrique,2,413,413,413,413,413,413
essence,1,844,844,844,844,844,844


Nous remarquons que la variable "product_type" nous donne la "garantie". Nous allons donc supprimer la variable "garantie" qui n'était pas présente initialement.

#### 4.2.2. Product type & quality

In [13]:
mower_by_type_by_quality = mower_df.groupby(["product_type", "quality"])
mower_by_type_by_quality.count()

capacity  failure_rate   id  price  warranty  \
product_type quality                                                 
auto-portee  Hight          16            16   16     16        16   
             Low            96            96   96     96        96   
             Medium         30            30   30     30        30   
electrique   Hight          44            44   44     44        44   
             Low           287           287  287    287       287   
             Medium         82            82   82     82        82   
essence      Hight          73            73   73     73        73   
             Low           588           588  588    588       588   
             Medium        183           183  183    183       183   

                      attractiveness  
product_type quality                  
auto-portee  Hight                16  
             Low                  96  
             Medium               30  
electrique   Hight                44  
             Low                 287  
             Medium               82  
essence      Hight                73  
             Low                 588  
             Medium              183

#### 4.2.2. Product type & quality

In [14]:
mower_by_quality_by_warranty = mower_df.groupby(["quality", "warranty"])
mower_by_quality_by_warranty.count()

capacity  failure_rate   id  price  product_type  \
quality warranty                                                     
Hight   1               73            73   73     73            73   
        2               44            44   44     44            44   
        3               16            16   16     16            16   
Low     1              588           588  588    588           588   
        2              287           287  287    287           287   
        3               96            96   96     96            96   
Medium  1              183           183  183    183           183   
        2               82            82   82     82            82   
        3               30            30   30     30            30   

                  attractiveness  
quality warranty                  
Hight   1                     73  
        2                     44  
        3                     16  
Low     1                    588  
        2                    287  
        3                     96  
Medium  1                    183  
        2                     82  
        3                     30

In [15]:
# suppression de la variable "warranty"
mower_df.drop(labels="warranty", axis=1, inplace=True)

## 5. Traitement des variables catégorielles

In [16]:
mowerByQuality = mower_df.groupby("quality")
mowerByQuality.id.count()

quality
Hight     133
Low       971
Medium    295
Name: id, dtype: int64

Si l'on regarde la répartition des tondeuses par qualité, on remarque qu'elle est très disparate, il faut donc **binariser** nos variables catégorielles.

In [17]:
# Binarisation des variables catégorielles
mower_cat = pd.get_dummies(mower_df)
mower_cat

,capacity,failure_rate,id,price,attractiveness,product_type_auto-portee,product_type_electrique,product_type_essence,quality_Hight,quality_Low,quality_Medium
0,10.492623,0.042528,18377,828.802103,0.650648,1.0,0.0,0.0,0.0,1.0,0.0
1,20.273730,0.037896,19363,882.894916,0.699792,1.0,0.0,0.0,0.0,1.0,0.0
2,20.190457,0.039111,19641,919.355522,0.632129,1.0,0.0,0.0,0.0,1.0,0.0
3,27.220131,0.041303,18728,817.085097,0.810730,1.0,0.0,0.0,0.0,1.0,0.0
4,16.370911,0.037938,18722,844.581465,0.757614,1.0,0.0,0.0,0.0,0.0,1.0
5,19.548213,0.035428,18268,905.297503,0.777051,1.0,0.0,0.0,0.0,0.0,1.0
6,12.242635,0.035492,18448,930.615022,0.766056,1.0,0.0,0.0,1.0,0.0,0.0
7,8.676314,0.042066,18943,818.075591,0.541318,1.0,0.0,0.0,0.0,1.0,0.0
8,15.264180,0.038804,19640,816.774028,0.620278,1.0,0.0,0.0,0.0,1.0,0.0
9,16.190545,0.034045,19559,814.799650,0.583117,1.0,0.0,0.0,0.0,1.0,0.0


In [18]:
# On enregistre la dataframe au format csv.
mower_cat.to_csv(path_or_buf=path_out, sep=',', index=False)